In [1]:
!sudo apt-get install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2build2).
0 upgraded, 0 newly installed, 0 to remove and 73 not upgraded.


In [2]:
import cv2
import numpy as np
import cv2
import os
from PIL import Image
import pytesseract
import matplotlib.pyplot as plt
%matplotlib inline

In [11]:
map_data = {
    "map_265": {
        "value1":{
            "color": (1,2,3),  # BGR format
            "lower_bound": 10,
            "upper_bound": 40,
        }
    },
}

def add_map(map_id, value_id, color, lower_bound, upper_bound):
    map_data[map_id][value_id] = {
        "color": color,
        "lower_bound": lower_bound,
        "upper_bound": upper_bound
    }

In [ ]:
# Argument Parser
args = {"image": "/kaggle/input/dv-pe/accuracy_images/map_287.png", "pre_processor": "thresh"}  # Replace with your image path

# print(map_name)
# Reading the image with text
images = cv2.imread(args["image"])
image_rgb = cv2.cvtColor(images, cv2.COLOR_BGR2RGB)

# Converting to grayscale image
gray = cv2.cvtColor(images, cv2.COLOR_BGR2GRAY)

# performing preprocessing techniques of threshing and/or blurring.
if args["pre_processor"] == "thresh":
    gray = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]
if args["pre_processor"] == "blur":
    gray = cv2.medianBlur(gray, 3)

# adding image to memory
filename = "{}.png".format(os.getpid())
cv2.imwrite(filename, gray)
# text = pytesseract.image_to_string(Image.open(filename))
# print(text)

def num_there(s):
    return any(i.isdigit() for i in s)

data = pytesseract.image_to_data(Image.open(filename), output_type=pytesseract.Output.DICT)

# Looping over the detected text and the bounding boxes
for i in range(len(data["text"])):
    value_id = 1
    if data["text"][i].strip() != "":
        x, y, w, h = data["left"][i], data["top"][i], data["width"][i], data["height"][i]
        # cv2.rectangle(images, (x, y), (x + w, y + h), (0, 255, 0), 2)

        text = data["text"][i]
        # print(f"Text: {text}, Bounding Box: {x}, {y}, {w}, {h}")

        if num_there(text)==1:
            # print(type(text))
            # print(f"Text: {text}, Bounding Box: {x}, {y}, {w}, {h}")
            # cv2.rectangle(images, (x, y), (x + w, y + h), (0, 255, 0), 2)
            
            x1 = x
            x2 = x
            y1 = y + h // 2
            y2 = y + h // 2 - 1
            
            while all(images[y1, x1]==images[y2, x2]):
                x2-=1
            x2-=10

            color = images[y2,x2]
            # print(color)
            # cv2.rectangle(images, (x2, y2), (x2+5, y2+5), (0, 255, 0), 2)

            values = text.split("-")
            # print(values)
            
            value_id += 1
            lower_bound = values[0]
            upper_bound = values[1]
            
            blue, green, red = color
            print(f"Blue: {blue}, Green: {green}, Red: {red}")
            
            # add_map(map_id, value_id, (blue, green, red), lower_bound, upper_bound)
            # print(map_data)
            
            map_name = "map_287"
            if map_name in map_data:
                map_data[map_name][value_id] = {
                    "color": (blue, green, red),
                    "lower_bound": lower_bound,
                    "upper_bound": upper_bound,
                }
            else:
                map_data[map_name] = {
                    value_id: {
                        "color": (blue, green, red),
                        "lower_bound": lower_bound,
                        "upper_bound": upper_bound,
                    }
                }
                
            print(map_data)

            # color = images[y,x-25] # have to edit this to get the exact colour.
            # print(f'Color at ({x}, {y}): {color}')

os.remove(filename)
plt.imshow(cv2.cvtColor(images, cv2.COLOR_BGR2RGB))